In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'fig_designs-supp'
cache_path = rd.rootdir/'output'/'fig_overview'/'data.gzip'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(metadata_path, style='designs')
data = data.merge(metadata, how='left', on='construct')
display(data)

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
designs_palette = metadata_dict['color']
designs_markers = metadata_dict['markers']

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UT')]

In [ ]:
# Bin data and calculate statistics
df_quantiles, stats, _, fits = base.calculate_bins_stats(df, num_bins=20)
stats = stats.merge(metadata, how='left', on='construct')
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

Load iPSC data (`data_ipsc`)

In [ ]:
exp83_5_path = base_path/'2024.04.29_exp83.5'/'export'

plates = pd.DataFrame({
    'data_path': [exp83_5_path],
    
    'yaml_path': [exp83_5_path/'exp83.5_wells.yaml'],
    
    'biorep': [1],
    
    'exp': ['exp83.5']
})

cache_path = rd.rootdir/'output'/'fig_designs'/'data_ipsc.gzip'

# Load data
data_ipsc = pd.DataFrame()
if cache_path.is_file(): data_ipsc = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data_ipsc = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_ipsc = data_ipsc[data_ipsc[c]>0]
    
    data_ipsc.dropna(inplace=True)
    data_ipsc.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
data_ipsc = data_ipsc.merge(metadata, how='left', on='construct')
display(data_ipsc)

In [ ]:
# Gate cells
gates_ipsc = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates_ipsc[channel] = data_ipsc[data_ipsc['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_ipsc.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates_ipsc.sort_values(['exp'], inplace=True)
gates_ipsc['marker'] = 'mGL-A'
gates_ipsc['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data_ipsc = data_ipsc.groupby('exp')[data_ipsc.columns].apply(lambda x: base.gate_data(x,gates_ipsc))
data_ipsc.reset_index(inplace=True, drop=True)
df_ipsc = data_ipsc[(data_ipsc['expressing']) & (data_ipsc['construct']!='UT')]

In [ ]:
# Bin data and calculate statistics
df_quantiles_ipsc, stats_ipsc, _, fits_ipsc = base.calculate_bins_stats(df_ipsc, num_bins=20)
stats_ipsc = stats_ipsc.merge(metadata, how='left', on='construct')
fits_ipsc = fits_ipsc.merge(metadata, how='left', on='construct')

Load data from 293T, MEF lenti infections (`data_lenti`)

In [ ]:
base_path_1 = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.04.05_exp89'/'export'
base_path_2 = rd.datadir/'instruments'/'data'/'attune'/'chris'/'2024.06.02-exp95-lenti-miR-iFFL'/'export'
plate_list = ['_'.join(x) for x in zip(
        ['plate'+str(i) for i in range(1,10)], 
        (['293T']*3 + ['MEF2A']*3 + ['MEF8A']*3),
        ['P9','P14','P15']*3
    )]

plates = pd.DataFrame({
    'data_path': [base_path_1/'293T_control', 
                  base_path_1/'293T_plate1', base_path_1/'293T_plate2', base_path_1/'293T_plate3',
                  base_path_1/'MEF_3_plate1', 
                  base_path_1/'MEF_4-1_plate1', base_path_1/'MEF_4-1_plate2', base_path_1/'MEF_4-1_plate3'] +
                 [base_path_2/p for p in plate_list],
    'yaml_path': [base_path_1/'kasey_yaml2'/'plate_control.yaml', 
                  base_path_1/'kasey_yaml2'/'plate01.yaml', base_path_1/'kasey_yaml2'/'plate02.yaml', base_path_1/'kasey_yaml2'/'plate03.yaml',
                  base_path_1/'kasey_yaml2'/'mef_3_plate01.yaml', 
                  base_path_1/'kasey_yaml2'/'mef_4-1_plate01.yaml', base_path_1/'kasey_yaml2'/'mef_4-1_plate02.yaml', base_path_1/'kasey_yaml2'/'mef_4-1_plate03.yaml'] +
                 [base_path_2/(p+'_metadata.yaml') for p in plate_list],
})

cache_path = rd.rootdir/'output'/'fig_designs'/'data_lenti.gzip'

# Load data
data_lenti = pd.DataFrame()
if cache_path.is_file(): data_lenti = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data_lenti = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data_lenti = data_lenti[data_lenti[c]>0]
    
    data.dropna(inplace=True)
    data_lenti.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
data_lenti = data_lenti.merge(metadata, how='left', on='construct')
data_lenti['cell'] = data_lenti['cell_type'].apply(lambda x: x.split('-')[0])
data_lenti['exp'] = data_lenti['cell_type'] + '_' + data_lenti['virus_batch']

def map_biorep(df):
    biorep_map = {val:i for i,val in enumerate(df['exp'].unique())}
    d = df.copy()
    d['biorep'] = d['exp'].map(biorep_map)
    return d

data_lenti = data_lenti.groupby('cell')[data_lenti.columns].apply(map_biorep).reset_index(drop=True)
display(data_lenti)

In [ ]:
# Gate cells
gates_lenti = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates_lenti[channel] = data_lenti[(data_lenti['virus_dilution']==0)].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates_lenti.reset_index(inplace=True)

# Add missing gates
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P10'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P14_'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['293T_P16'] + list(gates_lenti.loc[gates_lenti['exp']=='293T_na', channel_list].mean().values) 
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-3_P10'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P10'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P14'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)
gates_lenti.loc[len(gates_lenti.index)] = ['MEF-4-1_P16'] + list(gates_lenti.loc[gates_lenti['exp'].str.contains('MEF'), channel_list].mean().values)

# Indicate which channels are relevant for each experiment
gates_lenti.sort_values(['exp'], inplace=True)
gates_lenti['marker'] = 'mGL-A'
gates_lenti['output'] = 'mRuby2-A'

# Gate data by marker expression
data_lenti = data_lenti.groupby(['cell_type','virus_batch'])[data_lenti.columns].apply(lambda x: base.gate_data(x,gates_lenti))
data_lenti.reset_index(inplace=True, drop=True)
df_lenti = data_lenti[(data_lenti['expressing']) & (data_lenti['virus_dilution']!=0)]

In [ ]:
# Bin data and calculate statistics
df_lenti['output'] = df_lenti['output'].astype(float)
df_lenti['marker'] = df_lenti['marker'].astype(float)
df_quantiles_lenti, stats_lenti, _, fits_lenti = base.calculate_bins_stats(df_lenti, by=['exp','cell','biorep','construct','dox','virus_dilution'], num_bins=20)
stats_lenti = stats_lenti.merge(metadata, how='left', on='construct')
fits_lenti = fits_lenti.merge(metadata, how='left', on='construct')

In [ ]:
# Create a second palette for regular tuning
metadata2 = base.get_metadata(metadata_path, style='tuning')
metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
main_palette = metadata_dict2['color']
main_markers = metadata_dict2['markers']

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,10))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 2, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=([1]*5),)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,0]),
    'B': fig.add_subfigure(fig_gridspec[1,0]),
    'C': fig.add_subfigure(fig_gridspec[2,0]),
    'D': fig.add_subfigure(fig_gridspec[3,0]),
    'E': fig.add_subfigure(fig_gridspec[4,0]),
    'F': fig.add_subfigure(fig_gridspec[0,1]),
    'G': fig.add_subfigure(fig_gridspec[1,1]),
    'H': fig.add_subfigure(fig_gridspec[2,1]),
    'I': fig.add_subfigure(fig_gridspec[3,1]),
    'J': fig.add_subfigure(fig_gridspec[4,1]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
xlim = (2e2,6e4)
ylim = (2e1,1e5)

fig_path = output_path/'fig_designs-supp.pdf'
fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.8, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# EF1a miR controls
miR_order = ['none', 'miR.FF5', 'miR.FF4', 'miRE.FF5', 'miRE.FF4',]
plot_df = stats[((((stats['group']=='miR')) & (stats['miR_loc']=='UTR')) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
plot_df2 = fits[((((fits['group']=='miR')) & (fits['miR_loc']=='UTR')) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['miR'] = plot_df2['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['construct'].unique())-0.5)
ylim = (2e1,8e4)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    sns.despine(ax=ax)
    ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.42, bottom=0.5, top=0.8, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# EF1a miR controls
ts_order = ['none','FF3x1','FF4x1','FF5x1','FF6x1']
plot_df = stats[((((stats['group']=='ts5')) & (stats['ts_num']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
plot_df2 = fits[((((fits['group']=='ts5')) & (fits['ts_num']==1)) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['ts'] = plot_df2['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
xlim = (-0.5, len(plot_df['ts'].unique())-0.5)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    sns.despine(ax=ax)
    ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
def designs_stats_plots(axes, plot_df, plot_df2):

    xlim = (-0.5, len(plot_df['design'].unique())-0.5)

    # stat gmean
    ax = axes[0]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log',)
    marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
    ax.axhline(marker_baseline, color='black', ls=':')
    ax.minorticks_off()

    # stat std
    ax = axes[1]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Std.', xlim=xlim, xlabel='design', ylabel='', yscale='log',)

    # slope
    ax = axes[2]
    for construct, group in plot_df2.groupby('construct'):
        sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=(0,1.2),
        yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

    for ax in axes: sns.despine(ax=ax)

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.9, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

plot_df = stats[(stats['miR']=='miR.FF4') & (stats['group']=='controller') & (stats['ts_num']==1) &
                       (stats['promoter']=='EF1a')]
plot_df2 = fits[(fits['miR']=='miR.FF4') & (fits['group']=='controller') & (fits['ts_num']==1) &
                       (fits['promoter']=='EF1a')]

designs_stats_plots(axes, plot_df, plot_df2)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.9, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

plot_df = stats[(stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['ts_num']==1) & 
                       (stats['ts'].isin(['FF4x1','FF5x1','FF6x1'])) & (stats['promoter']=='EF1a')]
plot_df2 = fits[(fits['miR']=='miR.FF5') & (fits['group']=='controller') & (fits['ts_num']==1) &
                       (fits['ts'].isin(['FF4x1','FF5x1','FF6x1'])) & (fits['promoter']=='EF1a')]

designs_stats_plots(axes, plot_df, plot_df2)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.9, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

plot_df = stats[(stats['miR']=='miRE.FF5') & (stats['group']=='controller') & (stats['ts_num']==1) &
                       (stats['promoter']=='EF1a')]
plot_df2 = fits[(fits['miR']=='miRE.FF5') & (fits['group']=='controller') & (fits['ts_num']==1) &
                       (fits['promoter']=='EF1a')]

designs_stats_plots(axes, plot_df, plot_df2)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.8, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# iPSC controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats_ipsc[~stats_ipsc['group'].isin(['controller','marker']) & (stats_ipsc['promoter']=='EF1a')].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

plot_df2 = fits_ipsc[~fits_ipsc['group'].isin(['controller','marker']) & (fits_ipsc['promoter']=='EF1a')].copy()
plot_df2['group'] = plot_df2['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df2.sort_values(['group','ts'], inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
num_groups = 3
xtick_locs = [0, 1+buffer, 2+buffer] + [i+buffer*2 for i in range(3,7)]
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
construct_loc = {k:v for k,v in zip(plot_df2['construct'].unique(), xtick_locs)}
plot_df2['construct_loc'] = plot_df2['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
scatter_kwargs2 = dict(s=4, jitter=0.3, linewidth=0.5, edgecolor='white', native_scale=True)
xlim = (-0.5, plot_df['construct_loc'].max()+0.5)

# adjust markers
m.loc[(m['miR_loc']=='CDS') | (m['ts_loc']=='3\''), 'markers'] = 'D'
m_dict = m.set_index('construct').to_dict('dict')
comb_markers = m_dict['markers']

# make xticklables
def get_label(df):
    group = df['group'].unique()[0]
    d = df.copy()
    col_map = {'base': 'group', 'miR': 'miR', 'ts3': 'ts', 'ts5': 'ts'}
    d['label'] = d[col_map[group]]
    return d

m = m.groupby('group')[m.columns].apply(get_label).reset_index(drop=True)


# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs, ylim=(3e4,3e5))

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='', xticks=xtick_locs,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    sns.despine(ax=ax)

    # add shaded region for miR-only constructs
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    rd.plot.generate_xticklabels(m.drop_duplicates('construct_loc'), 'construct_loc', ['label'], annotate=False, ax=ax)
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.9, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))


plot_df = stats_ipsc[(stats_ipsc['group'].isin(['controller','base'])) & (stats_ipsc['promoter']=='CAG')]
plot_df2 = fits_ipsc[(fits_ipsc['group'].isin(['controller','base'])) & (fits_ipsc['promoter']=='CAG')]

# stat gmean
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_gmean', hue='construct', palette=main_palette,
                  legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_std', hue='construct', palette=main_palette,
                  legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log',)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='slope', hue='construct', palette=main_palette,
                  legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    ax.set_xticklabels([ts_label[x.get_text()] for x in ax.get_xticklabels()])
    sns.despine(ax=ax)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.8, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# iPSC controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats_lenti[~stats_lenti['group'].isin(['controller','marker']) & (stats_lenti['cell']=='293T') &
                      (stats_lenti['dox']==1000) & (stats_lenti['virus_dilution']==1)].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

plot_df2 = fits_lenti[~fits_lenti['group'].isin(['controller','marker']) & (fits_lenti['cell']=='293T') & 
                      (fits_lenti['dox']==1000) & (fits_lenti['virus_dilution']==1)].copy()
plot_df2['group'] = plot_df2['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df2.sort_values(['group','ts'], inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
num_groups = 3
xtick_locs = [0, 1+buffer, 2+buffer] + [i+buffer*2 for i in range(3,7)]
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
construct_loc = {k:v for k,v in zip(plot_df2['construct'].unique(), xtick_locs)}
plot_df2['construct_loc'] = plot_df2['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
scatter_kwargs2 = dict(s=4, jitter=0.3, linewidth=0.5, edgecolor='white', native_scale=True)
xlim = (-0.5, plot_df['construct_loc'].max()+0.5)

# adjust markers
m.loc[(m['miR_loc']=='CDS') | (m['ts_loc']=='3\''), 'markers'] = 'D'
m_dict = m.set_index('construct').to_dict('dict')
comb_markers = m_dict['markers']

# make xticklables
def get_label(df):
    group = df['group'].unique()[0]
    d = df.copy()
    col_map = {'base': 'group', 'miR': 'miR', 'ts3': 'ts', 'ts5': 'ts'}
    d['label'] = d[col_map[group]]
    return d

m = m.groupby('group')[m.columns].apply(get_label).reset_index(drop=True)


# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='', xticks=xtick_locs,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    sns.despine(ax=ax)

    # add shaded region for miR-only constructs
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    rd.plot.generate_xticklabels(m.drop_duplicates('construct_loc'), 'construct_loc', ['label'], annotate=False, ax=ax)
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['I']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.8, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# iPSC controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats_lenti[~stats_lenti['group'].isin(['controller','marker']) & (stats_lenti['cell']=='MEF') &
                      (stats_lenti['dox']==1000) & (stats_lenti['virus_dilution']==1)].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

plot_df2 = fits_lenti[~fits_lenti['group'].isin(['controller','marker']) & (fits_lenti['cell']=='MEF') & 
                      (fits_lenti['dox']==1000) & (fits_lenti['virus_dilution']==1)].copy()
plot_df2['group'] = plot_df2['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df2.sort_values(['group','ts'], inplace=True)

plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
plot_df2['construct_loc'] = plot_df2['construct'].replace(construct_loc)


# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='', xticks=xtick_locs,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    sns.despine(ax=ax)

    # add shaded region for miR-only constructs
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    rd.plot.generate_xticklabels(m.drop_duplicates('construct_loc'), 'construct_loc', ['label'], annotate=False, ax=ax)
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['J']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.9, bottom=0.4, top=0.35, right=0.05)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.45))

plot_df = stats_lenti[(stats_lenti['group'].isin(['controller','base'])) & 
                      (stats_lenti['dox']==0) & (stats_lenti['virus_dilution']==1)].copy()
plot_df2 = fits_lenti[(fits_lenti['group'].isin(['controller','base'])) & 
                      (fits_lenti['dox']==0) & (fits_lenti['virus_dilution']==1)].copy()
plot_df.sort_values(['cell','design'], inplace=True)
plot_df2.sort_values(['cell','design'], inplace=True)
plot_df['pair'] = plot_df['cell'] + '_' + plot_df['design'].astype(str)
plot_df2['pair'] = plot_df2['cell'] + '_' + plot_df2['design'].astype(str)

xlim = (-0.5, len(plot_df['pair'].unique())-0.5)

# stat gmean
ax = axes[0]
for name, group in plot_df.groupby('pair', sort=False):
    sns.stripplot(data=group, x='pair', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=('X' if name.split('_')[1]==0 else 'o'), **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='design', ylabel='', yscale='log', ylim=(2e1,6e1))
marker_baseline = sp.stats.gmean(data_lenti.loc[((data_lenti['construct']=='UT') | (data_lenti['virus_dilution']==0)) & (data_lenti['cell']=='293T'), 'output'])
ax.axhline(marker_baseline, -0.5, 2.5, color='black', ls=':')
marker_baseline = sp.stats.gmean(data_lenti.loc[((data_lenti['construct']=='UT') | (data_lenti['virus_dilution']==0)) & (data_lenti['cell']=='MEF'), 'output'])
ax.axhline(marker_baseline, 2.5, 5.5, color='black', ls=':')

# slope
ax = axes[1]
for name, group in plot_df2.groupby('pair', sort=False):
    sns.stripplot(data=group, x='pair', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=('X' if name.split('_')[1]==0 else 'o'), **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='design', ylabel='', ylim=(-0.2,0.2))
ax.axhline(0, color='black', ls=':')

for ax in axes: 
    sns.despine(ax=ax)
    ax.axvspan(2.5, 5.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    rd.plot.generate_xticklabels(plot_df.drop_duplicates('pair'), 'pair', ['design'], annotate=False, ax=ax)
    ax.annotate('HEK293T', (1,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
    ax.annotate('MEF', (4,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
    ax.set_title(ax.get_title(), pad=10)

fig.savefig(rd.outfile(fig_path))